### 모델 평가를 할 때는 accuracy를 쓰지 말고 손실율/AUC/f-1 score을 사용
### 예측결과와 실제 결과를 비교한다면 accuracy

In [2]:
# 경고 메세지가 안나오게..
import warnings
warnings.filterwarnings('ignore')

# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold
from sklearn.model_selection import KFold           # 랜덤하게 섞어서 자를 수도 있고, 순서대로 자를 수도 있음
from sklearn.model_selection import StratifiedKFold # 결과 데이터의 비율이 최대한 균등하게 들어갈 수 있도록!

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# 다중분류를 위한 핫-윈 인코더
from keras.utils import to_categorical

# 저장된 딥러닝 모델을 복구하는 함수
from keras.models import load_model

# epoch마다 모델을 저장하는 함수
from keras.callbacks import ModelCheckpoint

# 더이상 성능 향상이 이루어지지 않는다면 조기 중단시킬 수 있는 함수
from keras.callbacks import EarlyStopping

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family']='Malgun Gothic'
# 맥용 plt.rcParams['font.family']='AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [6]:
# 데이터 입력
df1 = pd.read_csv('../../data/breast_cancer.csv')
df1.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,malignant
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,malignant
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,malignant
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,malignant
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,malignant


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              454 non-null    float64
 1   mean texture             454 non-null    float64
 2   mean perimeter           454 non-null    float64
 3   mean area                454 non-null    float64
 4   mean smoothness          454 non-null    float64
 5   mean compactness         454 non-null    float64
 6   mean concavity           454 non-null    float64
 7   mean concave points      454 non-null    float64
 8   mean symmetry            454 non-null    float64
 9   mean fractal dimension   454 non-null    float64
 10  radius error             454 non-null    float64
 11  texture error            454 non-null    float64
 12  perimeter error          454 non-null    float64
 13  area error               454 non-null    float64
 14  smoothness error         4

In [9]:
df1.isnull().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64

In [7]:
# 데이터 분류
x = df1.drop('target', axis=1)
y = df1['target']

display(x)
display(y)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,21.10,20.52,138.10,1384.0,0.09684,0.11750,0.15720,0.11550,0.1554,0.05661,...,25.68,32.07,168.20,2022.0,0.13680,0.3101,0.4399,0.22800,0.2268,0.07425
450,11.87,21.54,76.83,432.0,0.06613,0.10640,0.08777,0.02386,0.1349,0.06612,...,12.79,28.18,83.51,507.2,0.09457,0.3399,0.3218,0.08750,0.2305,0.09952
451,19.59,25.00,127.70,1191.0,0.10320,0.09871,0.16550,0.09063,0.1663,0.05391,...,21.44,30.96,139.80,1421.0,0.15280,0.1845,0.3977,0.14660,0.2293,0.06091
452,12.00,28.23,76.77,442.5,0.08437,0.06450,0.04055,0.01945,0.1615,0.06104,...,13.09,37.88,85.07,523.7,0.12080,0.1856,0.1811,0.07116,0.2447,0.08194


0      malignant
1      malignant
2      malignant
3      malignant
4      malignant
         ...    
449    malignant
450       benign
451    malignant
452       benign
453       benign
Name: target, Length: 454, dtype: object

In [10]:
# 문자열을 숫자로 변환한다

encoder1 = LabelEncoder()
encoder1.fit(y)
y = encoder1.transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [16]:
# 모델 설정
model = Sequential()
model.add(Dense(60, input_dim=30, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [17]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# 모델 저장 콜백
modelpath = 'models/{epoch}-{val_loss}.hdf5'
callback1 = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

In [24]:
# 중단 콜백
# patience : 값을 조절하면서 확인 해보기

callback2 = EarlyStopping(monitor='val_loss', patience=1000)

In [25]:
# 모델 실행
# 어차피 조기 중단 되니까 에포크 많이 주기
# batch_size가 데이터보다 커도 상관없음

model.fit(x, y, validation_split=0.2, epochs=100000, batch_size=500, callbacks=[callback1, callback2])

Epoch 1/100000
1/1 [==============================] - 0s 78ms/step - loss: 0.1705 - accuracy: 0.9366 - val_loss: 0.1788 - val_accuracy: 0.9560

Epoch 00001: val_loss did not improve from 0.17519
Epoch 2/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.1704 - accuracy: 0.9366 - val_loss: 0.1789 - val_accuracy: 0.9560

Epoch 00002: val_loss did not improve from 0.17519
Epoch 3/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.1702 - accuracy: 0.9366 - val_loss: 0.1791 - val_accuracy: 0.9560

Epoch 00003: val_loss did not improve from 0.17519
Epoch 4/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.1700 - accuracy: 0.9366 - val_loss: 0.1792 - val_accuracy: 0.9560

Epoch 00004: val_loss did not improve from 0.17519
Epoch 5/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.1699 - accuracy: 0.9366 - val_loss: 0.1795 - val_accuracy: 0.9560

Epoch 00005: val_loss did not improve from 0.17519
Epoch 6/100000
1/1 [=====

1/1 [==============================] - 0s 45ms/step - loss: 0.1645 - accuracy: 0.9366 - val_loss: 0.1731 - val_accuracy: 0.9560

Epoch 00039: val_loss improved from 0.17320 to 0.17307, saving model to models\39-0.17307345569133759.hdf5
Epoch 40/100000
1/1 [==============================] - 0s 63ms/step - loss: 0.1644 - accuracy: 0.9366 - val_loss: 0.1731 - val_accuracy: 0.9560

Epoch 00040: val_loss did not improve from 0.17307
Epoch 41/100000
1/1 [==============================] - 0s 74ms/step - loss: 0.1642 - accuracy: 0.9366 - val_loss: 0.1731 - val_accuracy: 0.9560

Epoch 00041: val_loss improved from 0.17307 to 0.17307, saving model to models\41-0.17307308316230774.hdf5
Epoch 42/100000
1/1 [==============================] - 0s 50ms/step - loss: 0.1641 - accuracy: 0.9366 - val_loss: 0.1728 - val_accuracy: 0.9560

Epoch 00042: val_loss improved from 0.17307 to 0.17281, saving model to models\42-0.1728116273880005.hdf5
Epoch 43/100000
1/1 [==============================] - 0s 38ms/st

1/1 [==============================] - 0s 38ms/step - loss: 0.1598 - accuracy: 0.9311 - val_loss: 0.1690 - val_accuracy: 0.9451

Epoch 00073: val_loss improved from 0.16905 to 0.16902, saving model to models\73-0.16902293264865875.hdf5
Epoch 74/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.1597 - accuracy: 0.9311 - val_loss: 0.1691 - val_accuracy: 0.9451

Epoch 00074: val_loss did not improve from 0.16902
Epoch 75/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.1595 - accuracy: 0.9311 - val_loss: 0.1690 - val_accuracy: 0.9451

Epoch 00075: val_loss improved from 0.16902 to 0.16897, saving model to models\75-0.16896666586399078.hdf5
Epoch 76/100000
1/1 [==============================] - 0s 50ms/step - loss: 0.1594 - accuracy: 0.9311 - val_loss: 0.1686 - val_accuracy: 0.9451

Epoch 00076: val_loss improved from 0.16897 to 0.16864, saving model to models\76-0.16864469647407532.hdf5
Epoch 77/100000
1/1 [==============================] - 0s 37ms/s

1/1 [==============================] - 0s 39ms/step - loss: 0.1550 - accuracy: 0.9311 - val_loss: 0.1649 - val_accuracy: 0.9451

Epoch 00109: val_loss improved from 0.16488 to 0.16487, saving model to models\109-0.16486771404743195.hdf5
Epoch 110/100000
1/1 [==============================] - 0s 47ms/step - loss: 0.1548 - accuracy: 0.9311 - val_loss: 0.1646 - val_accuracy: 0.9451

Epoch 00110: val_loss improved from 0.16487 to 0.16457, saving model to models\110-0.1645713448524475.hdf5
Epoch 111/100000
1/1 [==============================] - 0s 57ms/step - loss: 0.1547 - accuracy: 0.9311 - val_loss: 0.1643 - val_accuracy: 0.9451

Epoch 00111: val_loss improved from 0.16457 to 0.16432, saving model to models\111-0.16431789100170135.hdf5
Epoch 112/100000
1/1 [==============================] - 0s 48ms/step - loss: 0.1546 - accuracy: 0.9311 - val_loss: 0.1642 - val_accuracy: 0.9451

Epoch 00112: val_loss improved from 0.16432 to 0.16419, saving model to models\112-0.16418905556201935.hdf5
Ep


Epoch 00142: val_loss did not improve from 0.16064
Epoch 143/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.1504 - accuracy: 0.9339 - val_loss: 0.1606 - val_accuracy: 0.9451

Epoch 00143: val_loss improved from 0.16064 to 0.16062, saving model to models\143-0.1606181561946869.hdf5
Epoch 144/100000
1/1 [==============================] - 0s 42ms/step - loss: 0.1503 - accuracy: 0.9339 - val_loss: 0.1604 - val_accuracy: 0.9451

Epoch 00144: val_loss improved from 0.16062 to 0.16036, saving model to models\144-0.1603611707687378.hdf5
Epoch 145/100000
1/1 [==============================] - 0s 47ms/step - loss: 0.1501 - accuracy: 0.9339 - val_loss: 0.1601 - val_accuracy: 0.9451

Epoch 00145: val_loss improved from 0.16036 to 0.16012, saving model to models\145-0.16012027859687805.hdf5
Epoch 146/100000
1/1 [==============================] - 0s 40ms/step - loss: 0.1500 - accuracy: 0.9339 - val_loss: 0.1600 - val_accuracy: 0.9451

Epoch 00146: val_loss improved from 0.1601

...

1/1 [==============================] - 0s 38ms/step - loss: 0.0295 - accuracy: 0.9917 - val_loss: 0.0886 - val_accuracy: 0.9670

Epoch 02010: val_loss did not improve from 0.05662
Epoch 2011/100000
1/1 [==============================] - 0s 40ms/step - loss: 0.0290 - accuracy: 0.9945 - val_loss: 0.0889 - val_accuracy: 0.9560

Epoch 02011: val_loss did not improve from 0.05662
Epoch 2012/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.0290 - accuracy: 0.9945 - val_loss: 0.0992 - val_accuracy: 0.9560

Epoch 02012: val_loss did not improve from 0.05662
Epoch 2013/100000
1/1 [==============================] - 0s 40ms/step - loss: 0.0295 - accuracy: 0.9917 - val_loss: 0.0827 - val_accuracy: 0.9670

Epoch 02013: val_loss did not improve from 0.05662
Epoch 2014/100000
1/1 [==============================] - 0s 42ms/step - loss: 0.0303 - accuracy: 0.9890 - val_loss: 0.1109 - val_accuracy: 0.9560

Epoch 02014: val_loss did not improve from 0.05662
Epoch 2015/100000
1/1 [=====

1/1 [==============================] - 0s 39ms/step - loss: 0.0288 - accuracy: 0.9917 - val_loss: 0.1022 - val_accuracy: 0.9560

Epoch 02051: val_loss did not improve from 0.05662
Epoch 2052/100000
1/1 [==============================] - 0s 40ms/step - loss: 0.0284 - accuracy: 0.9945 - val_loss: 0.0893 - val_accuracy: 0.9670

Epoch 02052: val_loss did not improve from 0.05662
Epoch 2053/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.0281 - accuracy: 0.9945 - val_loss: 0.0975 - val_accuracy: 0.9560

Epoch 02053: val_loss did not improve from 0.05662
Epoch 2054/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.0283 - accuracy: 0.9945 - val_loss: 0.0913 - val_accuracy: 0.9560

Epoch 02054: val_loss did not improve from 0.05662
Epoch 2055/100000
1/1 [==============================] - 0s 37ms/step - loss: 0.0281 - accuracy: 0.9945 - val_loss: 0.0935 - val_accuracy: 0.9560

Epoch 02055: val_loss did not improve from 0.05662
Epoch 2056/100000
1/1 [=====

1/1 [==============================] - 0s 38ms/step - loss: 0.0275 - accuracy: 0.9945 - val_loss: 0.1137 - val_accuracy: 0.9560

Epoch 02092: val_loss did not improve from 0.05662
Epoch 2093/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.0289 - accuracy: 0.9917 - val_loss: 0.0828 - val_accuracy: 0.9670

Epoch 02093: val_loss did not improve from 0.05662
Epoch 2094/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.0304 - accuracy: 0.9917 - val_loss: 0.1282 - val_accuracy: 0.9560

Epoch 02094: val_loss did not improve from 0.05662
Epoch 2095/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.0319 - accuracy: 0.9945 - val_loss: 0.0791 - val_accuracy: 0.9780

Epoch 02095: val_loss did not improve from 0.05662
Epoch 2096/100000
1/1 [==============================] - 0s 40ms/step - loss: 0.0331 - accuracy: 0.9890 - val_loss: 0.1365 - val_accuracy: 0.9560

Epoch 02096: val_loss did not improve from 0.05662
Epoch 2097/100000
1/1 [=====

1/1 [==============================] - 0s 38ms/step - loss: 0.0259 - accuracy: 0.9945 - val_loss: 0.1029 - val_accuracy: 0.9560

Epoch 02133: val_loss did not improve from 0.05662
Epoch 2134/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.0259 - accuracy: 0.9945 - val_loss: 0.0960 - val_accuracy: 0.9560

Epoch 02134: val_loss did not improve from 0.05662
Epoch 2135/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.0260 - accuracy: 0.9945 - val_loss: 0.1036 - val_accuracy: 0.9560

Epoch 02135: val_loss did not improve from 0.05662
Epoch 2136/100000
1/1 [==============================] - 0s 37ms/step - loss: 0.0260 - accuracy: 0.9945 - val_loss: 0.0957 - val_accuracy: 0.9560

Epoch 02136: val_loss did not improve from 0.05662
Epoch 2137/100000
1/1 [==============================] - 0s 38ms/step - loss: 0.0259 - accuracy: 0.9945 - val_loss: 0.1023 - val_accuracy: 0.9560

Epoch 02137: val_loss did not improve from 0.05662
Epoch 2138/100000
1/1 [=====

1/1 [==============================] - 0s 42ms/step - loss: 0.0256 - accuracy: 0.9945 - val_loss: 0.0924 - val_accuracy: 0.9670

Epoch 02174: val_loss did not improve from 0.05662
Epoch 2175/100000
1/1 [==============================] - 0s 41ms/step - loss: 0.0261 - accuracy: 0.9945 - val_loss: 0.1193 - val_accuracy: 0.9560

Epoch 02175: val_loss did not improve from 0.05662
Epoch 2176/100000
1/1 [==============================] - 0s 40ms/step - loss: 0.0265 - accuracy: 0.9945 - val_loss: 0.0911 - val_accuracy: 0.9670

Epoch 02176: val_loss did not improve from 0.05662
Epoch 2177/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.0267 - accuracy: 0.9945 - val_loss: 0.1182 - val_accuracy: 0.9560

Epoch 02177: val_loss did not improve from 0.05662
Epoch 2178/100000
1/1 [==============================] - 0s 39ms/step - loss: 0.0262 - accuracy: 0.9945 - val_loss: 0.0943 - val_accuracy: 0.9670

Epoch 02178: val_loss did not improve from 0.05662
Epoch 2179/100000
1/1 [=====

In [27]:
# 학습 완료된 모델을 로딩한다
best_model = load_model('models/1185-0.056624993681907654.hdf5')
best_model

In [30]:
a = best_model.evaluate(x,y)[1]
print(a)

15/15 [==============================] - 0s 2ms/step - loss: 0.0656 - accuracy: 0.9758
0.9757709503173828


In [28]:
# 학습데이터로 예측결과 추출
pred = (best_model.predict(x) > 0.5).astype('int32')
pred

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
    

In [29]:
r1 = accuracy_score(y, pred)
r1

0.9757709251101322

In [31]:
# 예측할 데이터를 불러온다
df2 = pd.read_csv('../../data/breast_cancer_new.csv')
df2.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,12.62,17.15,80.62,492.9,0.08583,0.05430,0.02966,0.02272,0.1799,0.05826,...,14.34,22.15,91.62,633.5,0.1225,0.1517,0.18870,0.09851,0.3270,0.07330
1,13.38,30.72,86.34,557.2,0.09245,0.07426,0.02819,0.03264,0.1375,0.06016,...,15.05,41.61,96.69,705.6,0.1172,0.1421,0.07003,0.07763,0.2196,0.07675
2,11.63,29.29,74.87,415.1,0.09357,0.08574,0.07160,0.02017,0.1799,0.06166,...,13.12,38.81,86.04,527.8,0.1406,0.2031,0.29230,0.06835,0.2884,0.07220
3,13.21,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,0.05584,...,14.35,34.23,91.29,632.9,0.1289,0.1063,0.13900,0.06005,0.2444,0.06788
4,13.00,25.13,82.61,520.2,0.08369,0.05073,0.01206,0.01762,0.1667,0.05449,...,14.34,31.88,91.06,628.5,0.1218,0.1093,0.04462,0.05921,0.2306,0.06291


In [32]:
# 예측결과 추출
pred = (best_model.predict(df2) > 0.5).astype('int32')
pred

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
    

In [33]:
# 결과를 문자열로 복원
pred2 = encoder1.inverse_transform(pred)
pred2

array(['benign', 'malignant', 'benign', 'benign', 'benign', 'benign',
       'malignant', 'malignant', 'benign', 'benign', 'benign',
       'malignant', 'benign', 'benign', 'malignant', 'benign', 'benign',
       'benign', 'benign', 'benign', 'benign', 'benign', 'malignant',
       'benign', 'benign', 'malignant', 'benign', 'malignant', 'benign',
       'benign', 'benign', 'benign', 'benign', 'malignant', 'benign',
       'malignant', 'benign', 'malignant', 'malignant', 'benign',
       'benign', 'benign', 'benign', 'benign', 'malignant', 'malignant',
       'benign', 'malignant', 'benign', 'malignant', 'benign', 'benign',
       'benign', 'benign', 'benign', 'malignant', 'benign', 'benign',
       'malignant', 'benign', 'malignant', 'benign', 'malignant',
       'malignant', 'benign', 'benign', 'benign', 'malignant', 'benign',
       'benign', 'benign', 'benign', 'benign', 'benign', 'benign',
       'benign', 'benign', 'benign', 'benign', 'malignant', 'benign',
       'malignant', 'be

In [34]:
# 결과 데이터를 붙힌다
df2['target'] = pred2
df2.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,12.62,17.15,80.62,492.9,0.08583,0.05430,0.02966,0.02272,0.1799,0.05826,...,22.15,91.62,633.5,0.1225,0.1517,0.18870,0.09851,0.3270,0.07330,benign
1,13.38,30.72,86.34,557.2,0.09245,0.07426,0.02819,0.03264,0.1375,0.06016,...,41.61,96.69,705.6,0.1172,0.1421,0.07003,0.07763,0.2196,0.07675,malignant
2,11.63,29.29,74.87,415.1,0.09357,0.08574,0.07160,0.02017,0.1799,0.06166,...,38.81,86.04,527.8,0.1406,0.2031,0.29230,0.06835,0.2884,0.07220,benign
3,13.21,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,0.05584,...,34.23,91.29,632.9,0.1289,0.1063,0.13900,0.06005,0.2444,0.06788,benign
4,13.00,25.13,82.61,520.2,0.08369,0.05073,0.01206,0.01762,0.1667,0.05449,...,31.88,91.06,628.5,0.1218,0.1093,0.04462,0.05921,0.2306,0.06291,benign


In [35]:
df2.to_csv('../../data/breast_cancer_deep_result.csv')